<a href="https://colab.research.google.com/github/lakshitgosain/Named_Entity_Recognition/blob/main/Named_Entity_Recognition_Lakshit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from google.colab import files
%pip install tensorflow

In [2]:
uploaded_file=files.upload()

Saving NER_Dataset.csv to NER_Dataset.csv


In [3]:
import pandas as pd
df=pd.read_csv('NER_Dataset.csv',encoding='unicode_escape')

In [4]:
df.head(10)

,ï»¿Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


In [5]:
df['Tag'].value_counts()

O        887908
B-geo     37644
B-tim     20333
B-org     20143
I-per     17251
B-per     16990
I-org     16784
B-gpe     15870
I-geo      7414
I-tim      6528
B-art       402
B-eve       308
I-art       297
I-eve       253
B-nat       201
I-gpe       198
I-nat        51
Name: Tag, dtype: int64

In [6]:
df.rename(columns={'ï»¿Sentence #':'Sentence #'},inplace=True)

In [7]:
df.columns

Index(['Sentence #', 'Word', 'POS', 'Tag'], dtype='object')

We will need to edit the data to make it compatible with the Neural Network Training..We will get the tag2id and token(word)2ID and vice versa

In [8]:
from itertools import chain
def get_dict_map(data, token_or_tag):
  tok2idx={}
  idx2tok={}

  if token_or_tag=='token':
    vocab=list(set(data['Word'].to_list()))
  else:
    vocab=list(set(data['Tag'].to_list()))

  idx2tok={idx:tok for idx, tok in enumerate(vocab)}
  tok2idx={tok:idx for idx, tok in enumerate(vocab)}
  return tok2idx,idx2tok

token2idx, idx2token=get_dict_map(df,'token')
tag2idx, idx2tag=get_dict_map(df,'tag')

In [9]:
tag2idx,token2idx

({'B-art': 15,
  'B-eve': 10,
  'B-geo': 13,
  'B-gpe': 4,
  'B-nat': 5,
  'B-org': 3,
  'B-per': 2,
  'B-tim': 8,
  'I-art': 12,
  'I-eve': 14,
  'I-geo': 16,
  'I-gpe': 1,
  'I-nat': 7,
  'I-org': 6,
  'I-per': 0,
  'I-tim': 11,
  'O': 9},
 {'assure': 0,
  'disturbing': 1,
  'compatible': 2,
  'Barrett': 3,
  'Six-thousand': 4,
  'Mil': 5,
  'Rugova': 6,
  'Rodriguez': 7,
  'kingdoms': 8,
  'Kibo': 9,
  'Singur': 10,
  'confrontation': 11,
  '106.5': 12,
  'Sabbath': 13,
  'improving': 14,
  'Herzegovina': 15,
  'Odinga': 16,
  'Ivanovo': 17,
  'interactive': 18,
  'Disney': 19,
  'Esquisabel': 20,
  'crops': 21,
  'homing': 22,
  'KIBAKI': 23,
  'predominate': 24,
  'Azov': 25,
  '84': 26,
  'Gref': 27,
  'Nad-e-Ali': 28,
  'Shallah': 29,
  'values': 30,
  'spare': 31,
  '301': 32,
  'Chase': 33,
  'shadows': 34,
  'garnered': 35,
  'Siberian': 36,
  'Francesca': 37,
  'insurgency-related': 38,
  'English-speaking': 39,
  'hijackers': 40,
  'Togolese': 41,
  'snapshot': 42,
  'Multi

In [10]:
df['Word_idx']=df['Word'].map(token2idx)
df['Tag_idx']=df['Tag'].map(tag2idx)

In [11]:
df

,Sentence #,Word,POS,Tag,Word_idx,Tag_idx
0,Sentence: 1,Thousands,NNS,O,17689,9
1,NaN,of,IN,O,32207,9
2,NaN,demonstrators,NNS,O,5809,9
3,NaN,have,VBP,O,18758,9
4,NaN,marched,VBN,O,5601,9
...,...,...,...,...,...,...
1048570,NaN,they,PRP,O,22848,9
1048571,NaN,responded,VBD,O,1042,9
1048572,NaN,to,TO,O,24707,9
1048573,NaN,the,DT,O,30738,9


In [12]:
df.isna().sum()

Sentence #    1000616
Word                0
POS                 0
Tag                 0
Word_idx            0
Tag_idx             0
dtype: int64

In [13]:
df['Sentence #']

0          Sentence: 1
1                  NaN
2                  NaN
3                  NaN
4                  NaN
              ...     
1048570            NaN
1048571            NaN
1048572            NaN
1048573            NaN
1048574            NaN
Name: Sentence #, Length: 1048575, dtype: object

In [14]:
df_filled=df.fillna(method='ffill',axis=0)

In [15]:
data_grouped=df_filled.groupby(['Sentence #'],as_index=False)['Word','POS', 'Tag', 'Word_idx', 'Tag_idx'].agg(lambda x: list(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [16]:
data_grouped

,Sentence #,Word,POS,Tag,Word_idx,Tag_idx
0,Sentence: 1,"[Thousands, of, demonstrators, have, marched, ...","[NNS, IN, NNS, VBP, VBN, IN, NNP, TO, VB, DT, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo...","[17689, 32207, 5809, 18758, 5601, 26652, 4527,...","[9, 9, 9, 9, 9, 9, 13, 9, 9, 9, 9, 9, 13, 9, 9..."
1,Sentence: 10,"[Iranian, officials, say, they, expect, to, ge...","[JJ, NNS, VBP, PRP, VBP, TO, VB, NN, TO, JJ, J...","[B-gpe, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[13828, 28624, 34783, 22848, 3430, 24707, 1000...","[4, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ..."
2,Sentence: 100,"[Helicopter, gunships, Saturday, pounded, mili...","[NN, NNS, NNP, VBD, JJ, NNS, IN, DT, NNP, JJ, ...","[O, O, B-tim, O, O, O, O, O, B-geo, O, O, O, O...","[21301, 7408, 19693, 33696, 25985, 26007, 3256...","[9, 9, 8, 9, 9, 9, 9, 9, 13, 9, 9, 9, 9, 9, 3,..."
3,Sentence: 1000,"[They, left, after, a, tense, hour-long, stand...","[PRP, VBD, IN, DT, NN, JJ, NN, IN, NN, NNS, .]","[O, O, O, O, O, O, O, O, O, O, O]","[10198, 11839, 9012, 5824, 4918, 7274, 9495, 7...","[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]"
4,Sentence: 10000,"[U.N., relief, coordinator, Jan, Egeland, said...","[NNP, NN, NN, NNP, NNP, VBD, NNP, ,, NNP, ,, J...","[B-geo, O, O, B-per, I-per, O, B-tim, O, B-geo...","[8283, 261, 24237, 11752, 17434, 9572, 14001, ...","[13, 9, 9, 2, 0, 9, 8, 9, 13, 9, 4, 9, 4, 9, 9..."
...,...,...,...,...,...,...
47954,Sentence: 9995,"[Opposition, leader, Mir, Hossein, Mousavi, ha...","[NNP, NN, NNP, NNP, NNP, VBZ, VBN, PRP, VBZ, T...","[O, O, O, B-per, I-per, O, O, O, O, O, O, O, O...","[21065, 17595, 20330, 29546, 13888, 19894, 957...","[9, 9, 9, 2, 0, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ..."
47955,Sentence: 9996,"[On, Thursday, ,, Iranian, state, media, publi...","[IN, NNP, ,, JJ, NN, NNS, VBN, DT, NN, IN, DT,...","[O, B-tim, O, B-gpe, O, O, O, O, O, O, O, O, B...","[12604, 30368, 8908, 13828, 14973, 22236, 1936...","[9, 8, 9, 4, 9, 9, 9, 9, 9, 9, 9, 9, 3, 6, 9, ..."
47956,Sentence: 9997,"[Following, Iran, 's, disputed, June, 12, elec...","[VBG, NNP, POS, JJ, NNP, CD, NNS, ,, NNS, NNS,...","[O, B-geo, O, O, B-tim, I-tim, O, O, O, O, O, ...","[29412, 17232, 836, 6833, 5218, 12791, 7567, 8...","[9, 13, 9, 9, 8, 11, 9, 9, 9, 9, 9, 9, 9, 9, 9..."
47957,Sentence: 9998,"[Since, then, ,, authorities, have, held, publ...","[IN, RB, ,, NNS, VBP, VBN, JJ, NNS, IN, DT, VB...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[30670, 8946, 8908, 31770, 18758, 1745, 12004,...","[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ..."


In [17]:
from sklearn.model_selection import train_test_split

In [18]:
from keras.preprocessing.sequence import pad_sequences#We will need to pad the sequences to get them all of same length as the Network expects sequences of equal length

In [19]:
from keras.utils import to_categorical

In [20]:
def pad_train_test(data_grouped,df):
  len_token=len(list(set(df['Word'].to_list())))
  len_tag=len(list(set(df['Tag'].to_list())))

  tokens=data_grouped['Word_idx'].to_list()
  max_len=max([len(s) for s in tokens])
  pad_tokens=pad_sequences(tokens,max_len,padding='post', value=len_token-1,dtype='int32')
  tags=data_grouped['Tag_idx'].to_list()
  pad_tags=pad_sequences(tags,maxlen=max_len,padding='post',value=tag2idx['O'],dtype='int32')
  n_tags=len(tag2idx)
  pad_tags=[to_categorical(i, num_classes=n_tags) for i in pad_tags]

  tokens_,test_tokens,tags_,test_tags=train_test_split(pad_tokens,pad_tags,test_size=0.1)
  train_tokens,val_tokens,train_tags,val_tags=train_test_split(tokens_,tags_,test_size=0.25)

  print(
        'train_tokens length:', len(train_tokens),
        '\ntrain_tokens length:', len(train_tokens),
        '\ntest_tokens length:', len(test_tokens),
        '\ntest_tags:', len(test_tags),
        '\nval_tokens:', len(val_tokens),
        '\nval_tags:', len(val_tags)
  )

  return train_tokens,val_tokens,test_tokens,train_tags,val_tags,test_tags



In [21]:
train_tokens,val_tokens,test_tokens,train_tags,val_tags,test_tags=pad_train_test(data_grouped,df)

train_tokens length: 32372 
train_tokens length: 32372 
test_tokens length: 4796 
test_tags: 4796 
val_tokens: 10791 
val_tags: 10791


In [25]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,Bidirectional, TimeDistributed
from tensorflow.keras.utils import plot_model
from numpy.random import seed
seed(1)
tf.random.set_seed(2)

In [26]:
input_dim=len(list(set(df['Word'].to_list())))+1
output_dim=64
input_length=max([len(s) for s in data_grouped['Word_idx'].to_list()])
n_tags=len(tag2idx)

In [49]:
def bilstm_lstm_model():
  model=Sequential()

  model.add(Embedding(input_dim=input_dim,output_dim=output_dim,input_length=input_length))

  model.add(Bidirectional(LSTM(units=output_dim,return_sequences=True, dropout=0.2,recurrent_dropout=0.2),merge_mode='concat'))

  model.add(LSTM(units=output_dim,return_sequences=True,dropout=0.5, recurrent_dropout=0.5))

  model.add(TimeDistributed(Dense(n_tags, activation='relu')))

  model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
  model.summary()

  return model

In [52]:
def model_train(x,y,model):
  loss=[]
  for i in range(25):
    hist=model.fit(x,y,batch_size=1000,verbose=1, epochs=1, validation_split=0.2)

    loss.append(hist.history['loss'][0])
  return loss

In [53]:
results=pd.DataFrame()
model_bilstm_lstm=bilstm_lstm_model()
plot_model(model_bilstm_lstm)
results['with_add_lstm']=model_train(train_tokens,np.array(train_tags),model_bilstm_lstm)


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 104, 64)           2250624   
_________________________________________________________________
bidirectional_5 (Bidirection (None, 104, 128)          66048     
_________________________________________________________________
lstm_11 (LSTM)               (None, 104, 64)           49408     
_________________________________________________________________
time_distributed_5 (TimeDist (None, 104, 17)           1105      
Total params: 2,367,185
Trainable params: 2,367,185
Non-trainable params: 0
_________________________________________________________________
26/26 [==============================] - 25s 971ms/step - loss: 0.0909 - accuracy: 0.9693 - val_loss: 0.1096 - val_accuracy: 0.9696


In [56]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
text = nlp('Hi, My name is Lakshit Gosain \n I am from India \n I am working as a Data Scientist \n  at Protiviti')
displacy.render(text, style = 'ent', jupyter=True)